In [45]:
%matplotlib inline
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cross_validation import KFold
from sklearn.neighbors import KNeighborsClassifier

#My method slow method
def scaleData(data, column):
    mean = data[int(column)].mean()
    std = data[int(column)].std()
    
    for index,rows in data.iterrows():            
        meanminus = rows[int(column)] - mean
        newframe = meanminus/std
        data[int(column)].iloc[index] = newframe 
    return data
    
def scaleData(dataFrame,flag,tLabel):
    df = dataFrame.copy()

    for var in df.drop(tLabel,axis=1):
        mean = df[var].mean()
        std = df[var].std()
        l1 = (df[var].abs()).sum()

        if(flag == 1):
            df[var] = (df[var]-mean)/std
        else:
            df[var] = df[var]/l1

    return df   

diabetesDf = pd.read_csv('pima-indians-diabetes.data', header = None)


def crossValidation(k,dataFrame,order):
    #kf contains indices of instances in each fold - use KFold to split
    kf = KFold(len(dataFrame),k)
    sum_accuracy = 0

    #KFold returns a list containing the training instances and testing instances in each list item
    #will have 10 items in the list if using 10-fold CV
    for train, test in kf:
        training = dataFrame.iloc[train].copy()
        testingDF = dataFrame.iloc[test].copy()
        
        
        #training = training.drop(8, 1)
        
        training = scaleData(training,1,8)
        target = training.as_matrix(columns=[8])
        
        training = training.as_matrix(columns=[0,1,2,3,4,5,6,7])
        
        testingtarget = testingDF.as_matrix(columns=[8])
        testingDF = scaleData(testingDF, 1, 8)
        testing = testingDF.as_matrix(columns=[0,1,2,3,4,5,6,7])
        target = target.squeeze()
        
        knn_model = KNeighborsClassifier(n_neighbors=order)
        knn_model.fit(training, target)
            
        x = 0
        tP = 0
        tN = 0
        f = 0
        for list in testing:
            list = list.reshape(1,-1) 
            predictVal = ((knn_model.predict(list)))
            t = testingtarget[x]
            #print (predictVal, t)
            if predictVal == 1 & t == 1:
                tP = tP + 1  
                #print ("True Positive")
            elif predictVal == 1 & t == 0:
                tN = tN + 1
                #print ("True Negative")
            else:
                f = f + 1
            x = x+1
        #print (tN, tP, f, x)
        accuracy = (tN+tP)/x
        sum_accuracy = sum_accuracy + accuracy
    return sum_accuracy/k


print (crossValidation(10, diabetesDf, 1))
print (crossValidation(10, diabetesDf, 3))
print (crossValidation(10, diabetesDf, 5))

#p = knn_model.predict(X)
#My scaling method, very slow
#headers = training.dtypes.index
#for x in np.nditer(headers):
 #   training = scaleData(training, x)
#print (training)






#print (knn_model.predict(X))

0.6992822966507177
0.7395249487354751
0.7538448393711552
